In [1]:
# Prepare gpu for training
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    tf.config.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

tf.config.list_physical_devices("GPU")


from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')

2022-11-13 17:00:31.363665: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-13 17:00:31.437807: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-13 17:00:31.927309: W tensorflow/tsl/platform/default/dso_loader.cc:66] Could not load dynamic library 'libnvinfer.so.8'; dlerror: libnvinfer.so.8: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/chen/miniconda3/lib/:/home/chen/miniconda3/envs/tf/lib/
2022-11-13 17:00:31.927561: W tensorflow/tsl/plat

1 Physical GPUs, 1 Logical GPU
INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3070, compute capability 8.6


2022-11-13 17:00:32.363826: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2022-11-13 17:00:32.368612: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2022-11-13 17:00:32.368712: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [2]:
from scripts.experiment import run_experiment

In [5]:
generator, discriminator = run_experiment("scapes_640_320_cropped_zero_pad_limit_0_5", "dcgan_scapes_640_360", "dcgan_scapes_v1", num_examples_to_generate=4, epochs=1, colab=False)

Found 3 files belonging to 1 classes.


In [6]:
generator.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 102400)            10342400  
                                                                 
 reshape_1 (Reshape)         (None, 10, 10, 1024)      0         
                                                                 
 re_lu_6 (ReLU)              (None, 10, 10, 1024)      0         
                                                                 
 batch_normalization_8 (Batc  (None, 10, 10, 1024)     4096      
 hNormalization)                                                 
                                                                 
 conv2d_transpose_6 (Conv2DT  (None, 20, 20, 512)      13107712  
 ranspose)                                                       
                                                                 
 batch_normalization_9 (Batc  (None, 20, 20, 512)     

In [9]:
discriminator.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_11 (Conv2D)          (None, 180, 320, 64)      4864      
                                                                 
 re_lu_14 (ReLU)             (None, 180, 320, 64)      0         
                                                                 
 dropout_8 (Dropout)         (None, 180, 320, 64)      0         
                                                                 
 conv2d_12 (Conv2D)          (None, 90, 160, 128)      204928    
                                                                 
 re_lu_15 (ReLU)             (None, 90, 160, 128)      0         
                                                                 
 dropout_9 (Dropout)         (None, 90, 160, 128)      0         
                                                                 
 conv2d_13 (Conv2D)          (None, 45, 80, 256)      